In [1]:
import numpy as np

In [21]:
def fit(x_train, y_train):
    result = {}
    class_values = set(y_train)
    for current_class in class_values:
        result[current_class] = {}
        result["total_data"] = len(y_train)
        current_class_rows = (y_train == current_class)
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        num_features = x_train.shape[1]
        result[current_class]["total_count"] = len(y_train_current)
        for j in range(1, num_features+1):
            result[current_class][j-1] = {}
            all_possible_values = set(x_train[:, j-1])
            for current_value in all_possible_values:
                result[current_class][j-1][current_value] = (x_train_current[:, j-1] == current_value).sum()
                
    return result

In [22]:
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[current_class].keys()) - 1
    for j in range(1, num_features+1):
        xj = x[j-1]
        count_current_class_xj = dictionary[current_class][j-1][xj] + 1
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j-1].keys())
        current_xj_probability = np.log(count_current_class_xj) - np.log(count_current_class)
        output = output + current_xj_probability
    return output

In [23]:
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        p_current_class = probability(dictionary, x, current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [24]:
def predict(dictionary, x_test):
    y_pred = []
    for x in x_test:
            x_class = predictSinglePoint(dictionary, x)
            y_pred.append(x_class)
    return y_pred

In [25]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5 * second_limit
    for i in range(len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif (column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [26]:
from sklearn import datasets
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [27]:
for i in range(x.shape[-1]):
    x[:, i] = makeLabelled(x[:, i])

In [37]:
from sklearn import model_selection

x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.25)

In [38]:
dicttionary = fit(x_train, y_train)

In [39]:
y_pred = predict(dicttionary, x_test)

In [40]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.92      1.00      0.96        11
           2       1.00      0.93      0.97        15

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

[[12  0  0]
 [ 0 11  0]
 [ 0  1 14]]


In [47]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.92      1.00      0.96        11
           2       1.00      0.93      0.97        15

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

[[12  0  0]
 [ 0 11  0]
 [ 0  1 14]]
